In [3]:
from vpython import *

#-----------------------------------------FUNCTIONS FOR GENERAL USE-----------------------------------------

def normalize(angle):
    if angle >= 2 * pi or angle < 0:
        angle %= 2*pi
    return angle

def angle_quad(sine, cosine):
    if (sine > 0 and cosine > 0):
        return asin(sine)
    elif (sine > 0 and cosine < 0):
        return pi - asin(sine)
    elif (sine < 0 and cosine < 0):
        return pi - asin(sine)
    elif (sine < 0 and cosine > 0):
        return 2*pi + asin(sine)
    else:
        return 0

def cross_x(vector1, vector2):
    x = (vector1.y * vector2.z - vector1.z * vector2.y)
    y = (vector1.z * vector2.x - vector1.x * vector2.z)
    z = (vector1.x * vector2.y - vector1.y * vector2.x)
    product = vector(x, y, z)
    return product

#m1 = 3x3, m2 = 3x1
def multMat(m1, m2):
    m3 = []
    for i in range(0, len(m1)):
        n = m1[i][0] * m2[0] + m1[i][1] * m2[1] + m1[i][2] * m2[2]
        m3.append(n)
    return m3

def det2x2(matrix):
    return float(matrix[0][0] * matrix[1][1] - matrix[0][1] * matrix[1][0])


def solve2eq(coeff, const):
    B1 = []
    B2 = []
    for i in range (0, len(coeff)):
        rowB1 = []
        rowB2 = []
        for j in range(0, len(coeff[0])):
            rowB1.append(coeff[i][j])
            rowB2.append(coeff[i][j])
        B1.append(rowB1)
        B2.append(rowB2)
    for i in range(0, len(const)):
        B1[i][0] = const[i]
        B2[i][1] = const[i]
    x = det2x2(B1) / det2x2(coeff)
    y = det2x2(B2) / det2x2(coeff)
    return [x, y]


def det3x3(matrix):
    xminor = []
    yminor = []
    zminor = []
    
    for i in range(1, len(matrix)):
        rowxminor = []
        for j in range(1, len(matrix[0])):
            rowxminor.append(matrix[i][j])
        xminor.append(rowxminor)
    for i in range(1, len(matrix)):
        rowzminor = []
        for j in range(0, len(matrix[0])-1):
            rowzminor.append(matrix[i][j])
        zminor.append(rowzminor)
    for i in range(1, len(matrix)):
        rowyminor = []
        for j in range(0, len(matrix[0])):
            if j % 2 == 0:
                rowyminor.append(matrix[i][j])
        yminor.append(rowyminor)
        
    return matrix[0][0] * det2x2(xminor) - matrix[0][1] * det2x2(yminor) + matrix[0][2] * det2x2(zminor)


def solve3eq(coeff, const):
    B1 = []
    B2 = []
    B3 = []
    for i in range (0, len(coeff)):
        rowB1 = []
        rowB2 = []
        rowB3 = []
        for j in range(0, len(coeff[0])):
            rowB1.append(coeff[i][j])
            rowB2.append(coeff[i][j])
            rowB3.append(coeff[i][j])
        B1.append(rowB1)
        B2.append(rowB2)
        B3.append(rowB3)
    for i in range(0, len(const)):
        B1[i][0] = const[i]
        B2[i][1] = const[i]
        B3[i][2] = const[i]
    x = det3x3(B1) / det3x3(coeff)
    y = det3x3(B2) / det3x3(coeff)
    z = det3x3(B3) / det3x3(coeff)
    return [x, y, z]

#-----------------------------------------GAUSSIAN METHOD---------------------------------------------------

#------------------------------------0. correct for light travel time---------------------------------------

def lightTravel(ts, rho):
    c = (2.9989 * 10**3) * (6.68459 * 10**(-12)) #speed of light (AUs per second)
    tneg1corr = ts[0] - mag(rho[0]) / c
    t0corr = ts[1] - mag(rho[1]) / c
    t1corr = ts[2] - mag(rho[2]) / c
    return [tneg1corr, t0corr, t1corr]

#-----------------------------------------1. initial guess for r0 and rdot0---------------------------------

def rhohat(RA, DEC):
    x = cos(radians(DEC)) * cos(radians(RA))
    y = cos(radians(DEC)) * sin(radians(RA))
    z = sin(radians(DEC))
    return vector(x, y, z)

def aposAneg(ts):
    k = 0.01720209894
    tauneg1 = k * (ts[1] - ts[0])
    tau1 = k * (ts[2] - ts[1])
    tau0 = k * (ts[2] - ts[0]) 
    
    aneg = tau1 / tau0
    apos = tauneg1 / tau0 
    return [apos, aneg]
    
def rhomags(ts, RAs, DECs, Rs, aneg, apos):
    rhohatneg1 = rhohat(RAs[0], DECs[0])
    rhohat0 = rhohat(RAs[1], DECs[1])
    rhohat1 = rhohat(RAs[2], DECs[2])
    
    coeff = [[aneg * rhohatneg1.x, -rhohat0.x, apos * rhohat1.x],
            [aneg * rhohatneg1.y, -rhohat0.y, apos * rhohat1.y],
            [aneg * rhohatneg1.z, -rhohat0.z, apos * rhohat1.z]]
    const = [aneg * Rs[0].x - Rs[1].x + apos * Rs[2].x,
            aneg * Rs[0].y - Rs[1].y + apos * Rs[2].y,
            aneg * Rs[0].z - Rs[1].z + apos * Rs[2].z]
    
    return solve3eq(coeff, const)

def rhos(ts, RAs, DECs, Rs, aneg, apos):
    rhohatneg1 = rhohat(RAs[0], DECs[0])
    rhohat0 = rhohat(RAs[1], DECs[1])
    rhohat1 = rhohat(RAs[2], DECs[2])
    
    rhoneg1 = rhomags(ts, RAs, DECs, Rs, aneg, apos)[0] * rhohatneg1
    rho0 = rhomags(ts, RAs, DECs, Rs, aneg, apos)[1] * rhohat0
    rho1 = rhomags(ts, RAs, DECs, Rs, aneg, apos)[2] * rhohat1
    return [rhoneg1, rho0, rho1]
    
def r0rdot0(ts, Rs, RAs, DECs, aneg, apos):
    k = 0.01720209894
    tau0 = k * (ts[2] - ts[0])

    rneg1 = rhos(ts, RAs, DECs, Rs, aneg, apos)[0] - Rs[0]
    r0 = rhos(ts, RAs, DECs, Rs, aneg, apos)[1] - Rs[1]
    r1 = rhos(ts, RAs, DECs, Rs, aneg, apos)[2] - Rs[2]
    rdot0 = (r1 - rneg1) / tau0
    return [r0, rdot0]

#--------------------------------2. calculate new rneg1 and r1 with f- and g-series-------------------------

def taus(ts):
    k = 0.01720209894
    tauneg1 = k * (ts[0] - ts[1])
    tau1 = k * (ts[2] - ts[1])
    tau0 = k * (ts[2] - ts[0])
    return [tauneg1, tau0, tau1]

def f(tau, r0, rdot0): #f-series
    term1 = 1
    term2 = -tau**2 / (2 * mag(r0)**3)
    term3 = (dot(r0, rdot0) * tau**3) / (2 * mag(r0)**5)
    term4 = (tau**4 / 24) * ((3 / mag(r0)**3) * ((dot(rdot0, rdot0) / mag(r0)**2) - (1 / mag(r0)**3))
                            - (15 * dot(r0, rdot0)**2 / mag(r0)**7) + (1 / mag(r0)**6))
    return term1 + term2 + term3 + term4

def g(tau, r0, rdot0): #g-series
    term1 = tau
    term2 = -tau**3 / (6 * mag(r0)**3)
    term3 = (dot(r0, rdot0) * tau**4) / (4 * mag(r0)**5)
    return term1 + term2 + term3

def rneg1r1(r0, rdot0, tau):
    rneg1 = f(tau[0], r0, rdot0) * r0 + g(tau[0], r0, rdot0) * rdot0
    r1 = f(tau[2], r0, rdot0) * r0 + g(tau[2], r0, rdot0) * rdot0
    return [rneg1, r1]

#----------------------------3. calculate new r0 and rdot0 with new aneg and apos---------------------------

def Aneg(fneg1, f1, gneg1, g1):
    return g1 / (fneg1 * g1 - f1 * gneg1)

def Apos(fneg1, f1, gneg1, g1):
    return - gneg1 / (fneg1 * g1 - f1 * gneg1)

def bneg(fneg1, f1, gneg1, g1):
    return f1 / (f1 * gneg1 - fneg1 * g1)
    
def bpos(fneg1, f1, gneg1, g1):
    return - fneg1 / (f1 * gneg1 - fneg1 * g1)

def newrdot0(fneg1, f1, gneg1, g1, rneg1, r1):
    return bneg(fneg1, f1, gneg1, g1) * rneg1 + bpos(fneg1, f1, gneg1, g1) * r1

def newrs(rho, Rs):
    r0 = rho[1] - Rs[1]
    rneg1 = rho[0] - Rs[0]
    r1 = rho[2] - Rs[2]
    return [rneg1, r0, r1]

#----------------------------4. calculate new rhos for light travel time correction--------------------------

def newrhos(ts, RAs, DECs, Rs, fneg1, f1, gneg1, g1):
    aneg = Aneg(fneg1, f1, gneg1, g1)
    apos = Apos(fneg1, f1, gneg1, g1)
    return rhos(ts, RAs, DECs, Rs, aneg, apos)

#------------------------------------------------5. iterate!!!----------------------------------------------

def gauss(obs): #obs = list of obs1, obs2, obs3
    Rs = []
    ts = []
    RAs = []
    DECs = []
    for i in range(0, len(obs)):
        Rs.append(vector(obs[i][3], obs[i][4], obs[i][5]))
        ts.append(obs[i][0])
        RAs.append(obs[i][1] * 15.)
        DECs.append(obs[i][2])
    
    apos = aposAneg(ts)[0]
    aneg = aposAneg(ts)[1]
    rho = rhos(ts, RAs, DECs, Rs, aneg, apos) #set rhos equal to initial guesses of rhos
    
    r0 = r0rdot0(ts, Rs, RAs, DECs, aneg, apos)[0]
    rdot0 = r0rdot0(ts, Rs, RAs, DECs, aneg, apos)[1] #first guesses of r0 and rdot0
    
    oldr0 = vector(1, 1, 1)
    count = 0
    while (abs(mag(r0) - mag(oldr0)) > 10**(-9)):#while loop for precision
        #calculate new r0 and rdot0
        fneg1 = f(taus(ts)[0], r0, rdot0)
        f1 = f(taus(ts)[2], r0, rdot0) 
        gneg1 = g(taus(ts)[0], r0, rdot0)
        g1 = g(taus(ts)[2], r0, rdot0)
        
        oldr0 = r0
        
        rho = newrhos(ts, RAs, DECs, Rs, fneg1, f1, gneg1, g1)
        
        rs = newrs(rho, Rs)

        rdot0 = newrdot0(fneg1, f1, gneg1, g1, rs[0], rs[2])

        r0 = rs[1]
        print "r0", r0
        print "rdot0", rdot0
        count += 1
        
        #correct for light travel time
        #ts = lightTravel(ts, rho)
    print "count", count

    return [r0, rdot0, ts[1]] #ts[1] is the light-corrected time for the middle observation

#-----------------------------------------------LAPLACIAN METHOD--------------------------------------------

#-----------------------------1. initial guess for r0 and calculation of rhohats----------------------------

def initr0(ts, Rs, RAs, DECs, aneg, apos):
    k = 0.01720209894
    tau0 = k * (ts[2] - ts[0])

    rneg1 = rhos(ts, RAs, DECs, Rs, aneg, apos)[0] - Rs[0]
    r0 = rhos(ts, RAs, DECs, Rs, aneg, apos)[1] - Rs[1]
    r1 = rhos(ts, RAs, DECs, Rs, aneg, apos)[2] - Rs[2]
    return r0

#------------------------------------2. calculate rhohatdot0 and rhohatdotdot0------------------------------

def taus(ts):
    k = 0.01720209894
    tauneg1 = k * (ts[0] - ts[1])
    tau1 = k * (ts[2] - ts[1])
    tau0 = k * (ts[2] - ts[0])
    return [tauneg1, tau0, tau1]

def Rhohatdot0(taus, rhohats):
    return (taus[2]**2 * (rhohats[0] - rhohats[1]) - taus[0]**2 * (rhohats[2] - rhohats[1])) / (taus[0] * taus[1] * taus[2])

def Rhohatdotdot0(taus, rhohats):
    return -2 * ((taus[2] * (rhohats[0] - rhohats[1]) - taus[0] * (rhohats[2] - rhohats[1])) / (taus[0] * taus[1] * taus[2]))

#-----------------------------------------3. calculate new r0mag and rhomag---------------------------------

def rho0Mag(r0mag, rhohat0, rhohatdot0, rhohatdotdot0, R0):
    earthtosun = 1. / 328900.5
    A = (dot(cross_x(rhohat0, rhohatdot0), R0)) / (dot(cross_x(rhohat0, rhohatdot0), rhohatdotdot0))
    B = ((1 + earthtosun) / mag(R0)**3) * A
    return A / r0mag**3 - B

def r0Mag(rho0mag, rhohat0, R0):
    return sqrt(rho0mag**2 + mag(R0)**2 - 2 * rho0mag * dot(rhohat0, R0))

#-----------------------------------------------4. iterate!!!-----------------------------------------------

def r0rho0Mags(r0mag, rhohats, taus, R0):
    rhohatdot0 = Rhohatdot0(taus, rhohats)
    rhohatdotdot0 = Rhohatdotdot0(taus, rhohats)
    oldr0mag = 0
    while abs(r0mag - oldr0mag) > 10**(-9):
        oldr0mag = r0mag
        rho0mag = rho0Mag(r0mag, rhohats[1], rhohatdot0, rhohatdotdot0, R0)
        r0mag = r0Mag(rho0mag, rhohats[1], R0)
    return [r0mag, rho0mag]

#-------------------------------5. calculate final r0 and rdot0 from rhomagdot and Rdot---------------------

def Rhomagdot0(r0mag, rhohat0, rhohatdot0, rhohatdotdot0, R0):
    earthtosun = 1. / 328900.5
    A = (dot(cross_x(rhohat0, rhohatdotdot0), R0)) / (dot(cross_x(rhohat0, rhohatdot0), rhohatdotdot0))
    B = ((1 + earthtosun) / mag(R0)**3) * A
    return -0.5 * (A / r0mag**3 - B)

def RDot0(Rs, taus):
    return (Rs[2] - Rs[0]) / (taus[2] - taus[0])

def r0rdot02(r0mag, rho0mag, rhohat0, rhohatdot0, rhohatdotdot0, Rs, taus):
    rhomagdot0 = Rhomagdot0(r0mag, rhohat0, rhohatdot0, rhohatdotdot0, Rs[1])
    Rdot0 = RDot0(Rs, taus)
    r0 = rho0mag * rhohat0 - Rs[1]
    rdot0 = rhomagdot0 * rhohat0 + rho0mag * rhohatdot0 - Rdot0
    return [r0, rdot0]

#------------------------------------------6. put it all together------------------------------------------

def laplace(obs):
    Rs = []
    ts = []
    RAs = []
    DECs = []
    for i in range(0, len(obs)):
        Rs.append(vector(obs[i][3], obs[i][4], obs[i][5]))
        ts.append(obs[i][0])
        RAs.append(obs[i][1] * 15.)
        DECs.append(obs[i][2])
    
    apos = aposAneg(ts)[0]
    aneg = aposAneg(ts)[1]
    r0 = initr0(ts, Rs, RAs, DECs, aneg, apos) #first guesses of r0 and rdot0
    
    rhohats = []
    for i in range(0, len(RAs)):
        rhohats.append(rhohat(RAs[i], DECs[i]))
    
    rhohatdot0 = Rhohatdot0(taus(ts), rhohats)
    rhohatdotdot0 = Rhohatdotdot0(taus(ts), rhohats)
    
    mags = r0rho0Mags(mag(r0), rhohats, taus(ts), Rs[1])
    r0 = r0rdot02(mags[0], mags[1], rhohats[1], rhohatdot0, rhohatdotdot0, Rs, taus(ts))[0]
    rdot0 = r0rdot02(mags[0], mags[1], rhohats[1], rhohatdot0, rhohatdotdot0, Rs, taus(ts))[1]
    
    return [r0, rdot0, ts[1]]

#--------------------------------convert from equatorial to ecliptic coordinates----------------------------

def eqToEc(rEq):
    eps = radians(-23.439281)
    epsRot = [[1, 0, 0], [0, cos(eps), -sin(eps)], [0, sin(eps), cos(eps)]] 
    rEc = multMat(epsRot, rEq)
    return rEc

#-----------------------------------------CALCULATING ORBITAL ELEMENTS--------------------------------------

def H(r, rdot):
    return cross_x(r, rdot)

def A(r, rdot):
    vsquared = rdot.x**2 + rdot.y**2 + rdot.z**2
    return 1 / ( (2 / mag(r)) - vsquared )

def E(r, rdot):
    return sqrt( 1 - mag(H(r, rdot))**2 / A(r, rdot) )

def I(r, rdot):
    h = H(r, rdot)
    return normalize(atan( sqrt(h.x**2 + h.y**2) / h.z ))

def big_omega(r, rdot):
    h = H(r, rdot)
    cosine = -h.y / (mag(h) * sin(I(r, rdot)))
    sine = h.x / (mag(h) * sin(I(r, rdot)))
    return normalize(angle_quad(sine, cosine))

def U(r, rdot): #True longitude
    cosine = ( r.x * cos(big_omega(r, rdot)) + r.y * sin(big_omega(r, rdot)) ) / mag(r)
    sine = r.z / (mag(r) * sin(I(r, rdot)))
    return angle_quad(sine, cosine)

def V(r, rdot): #True anomaly
    e = E(r, rdot)
    a = A(r, rdot)
    h = H(r, rdot)
    rrdot = r.x * rdot.x + r.y * rdot.y + r.z * rdot.z
    sine = (a * (1 - e**2) / (e * mag(h))) * (rrdot / mag(r))
    cosine = (1 / e) * (a * (1 - e**2) / mag(r) - 1) 
    return angle_quad(sine, cosine)

def smallOmega(r, rdot):
    u = U(r, rdot)
    v = V(r, rdot)
    return normalize(u - v)

def COE(r, rdot): #Classical Orbital Elements
    return [A(r, rdot), E(r, rdot), degrees(I(r, rdot)), degrees(big_omega(r, rdot)), degrees(smallOmega(r, rdot)), degrees(V(r, rdot))]

#---------------------------------------ORBITAL DETERMINATION------------------------------------------------

def readFile(fileName):
    mFile = open(fileName, 'r')
    lines = []
    table = []
    for line in mFile:
        lines.append(line)
    for i in range(0, len(lines)):
        stripped = lines[i].strip()
        row = stripped.split()
        for i in range(0, len(row)):
            row[i] = float(row[i])
        table.append(row)
    return table #obs1, obs2, obs3

def orbit(obs):
    print "hi again again"
    r0rdot0t0 = gauss(obs)
    for i in range(0, 2):
        print "hi again again again"
        r0rdot0t0[i] = [r0rdot0t0[i].x, r0rdot0t0[i].y, r0rdot0t0[i].z]
        r0rdot0t0[i] = eqToEc(r0rdot0t0[i])
        r0rdot0t0[i] = vector(r0rdot0t0[i][0], r0rdot0t0[i][1], r0rdot0t0[i][2])
        print "hi again again again"
    
    coe = COE(r0rdot0t0[0], r0rdot0t0[1])
    print "Middle Observation:" + "\n\tposition vector =", r0rdot0t0[0]
    print "\tvelocity vector =", r0rdot0t0[1]
    print "\tlight-corrected time (JD)=", r0rdot0t0[2]
    print "\nClassical orbital elements:" + "\n\tsemi-major axis (a) =", coe[0]
    print "\teccentricity (e) =", coe[1]
    print "\tinclination (i) =", coe[2], u'\u00b0'
    print "\tlongitude of ascending node (big O) =", coe[3], u'\u00b0'
    print "\targument of perihelion (small O) =", coe[4], u'\u00b0'
    print "\ttrue longitude (v) =", coe[5], u'\u00b0'
    return [coe, r0rdot0t0]

#----------------------------------------------JACK-KNIFE-------------------------------------------------

def average(l):
    suml = 0.
    for i in range(0, len(l)):
        suml += l[i]
    return suml / len(l)

def averageVec(l):
    suml = vector(0, 0, 0)
    for i in range(0, len(l)):
        suml += l[i]
    return suml / len(l)

def error(l):
    N = len(l)
    avg = average(l)
    sigma = 0
    for i in range(0, len(l)):
        sigma += (l[i] - avg)**2
    return sqrt(sigma / (N * (N - 1)))

def jackknife(fileName):
    obs = readFile(fileName)
    data = []
    for h in range(0, len(obs)):
        for j in range(h + 1, len(obs)):
            for k in range(j + 1, len(obs)):
                data.append([obs[h], obs[j], obs[k]])
    
    a = []
    e = []
    I = []
    bigO = []
    smallO = []
    v = []
    r0 = []
    rdot0 = []
    t = []
    print "hi"
    for i in range(0, len(data)):
        print "hi again"
        coe = orbit(data[i])[0]
        a.append(coe[0])
        e.append(coe[1])
        I.append(coe[2])
        bigO.append(coe[3])
        smallO.append(coe[4])
        v.append(coe[5])
        
        r0rdot0t0 = orbit(data[i])[1]
        r0.append(r0rdot0t0[0])
        rdot0.append(r0rdot0t0[1])
        t.append(r0rdot0t0[2])
    
    print "Average time (JD):", average(t)
    print "Average position vector (AU):", averageVec(r0)
    print "Average velocity vector (AU):", averageVec(rdot0)
    
    print "\nClassical orbital elements:" + "\n\tsemi-major axis (a) =", average(a), "+-", error(a)
    print "\teccentricity (e) =", average(e), "+-", error(e)
    print "\tinclination (i) =", str(average(I)) + u'\u00b0', "+-", str(error(I)) + u'\u00b0'
    print "\tlongitude of ascending node (big O) =", str(average(bigO)) + u'\u00b0', "+-", str(error(bigO)) + u'\u00b0'
    print "\targument of perihelion (small O) =", str(average(smallO)) + u'\u00b0', "+-", str(error(smallO)) + u'\u00b0'
    print "\ttrue longitude (v) =", str(average(v)) + u'\u00b0', "+-", str(error(v)) + u'\u00b0'

#----------------------------------NUMERICAL INTEGRATION TO 100 YEARS IN THE FUTURE-------------------------
#CHANGES ACCELERATION ACCORDING TO FORCE OF GRAVITY
def acceleration(objpos):
    r = sqrt(objpos.x**2 + objpos.y**2 + objpos.z**2)
    aMag = 1. / r **2
    a = vec(objpos.x * -aMag/ r, objpos.y * -aMag/ r, objpos.z * -aMag/ r)
    return a

#MAKES SOLAR SYSTEM!!!
def integrate100(sunpos, earthpos, earthvel, asteroidpos, asteroidvel):
    objpos = [earthpos, asteroidpos]
    objvel = [earthvel * 58.1, asteroidvel]
    
    k = 0.01720209894

    t = 0 #INITIAL TIME
    dt = 0.1 #TIME INTERVAL
    max_t = 100 * 365.25 / k #100 years in the future in gaussian days

    min_dist = mag(earthpos) - mag(asteroidpos)
    min_dist_t = t

    sleep(0.5)
    while t < max_t:
        oneTime = 0
        for i in range(0, 2):
            if oneTime==0:
                a = acceleration(objpos[i])
                oneTime += 1
            aL = acceleration(objpos[i])
            objpos[i] += objvel[i] * dt + 0.5 * a * dt**2
            aR = acceleration(objpos[i])
            a = (aL + aR)/2.
            objvel[i] += a * dt

        if abs(mag(objpos[0]) - mag(objpos[1])) < min_dist:
            min_dist = abs(mag(objpos[0]) - mag(objpos[1]))
            min_dist_t = t

        t += dt
        rate(50000)

    return [min_dist, min_dist_t / k]

def closestEncounter(r0, rdot0, t):
    sunpos = vec(0, 0, 0)        
    
    print "Earth's position vector(ecliptic, JD = " + str(t) + "):"
    posx = input("\tx = ")
    posy = input("\ty = ")
    posz = input("\tz = ")
    
    print "Earth's velocity vector(ecliptic, JD = " + str(t) + "):"
    velx = input("\tx = ")
    vely = input("\ty = ")
    velz = input("\tz = ")
    
    earthpos = vec(posx, posy, posz) 
    earthvel = vec(velx, vely, velz)
    
    encounter = integrate100(sunpos, earthpos, earthvel, r0, rdot0)
    
    print "Closest encounter in the next 100 years: "
    print "\tdate/time (JD):", encounter[1]
    print "\tdistance (AU):", encounter[0]
    
    return [encounter, t]
    
#--------------------------------------JACK-KNIFE FOR NUMERICAL INTEGRATION-----------------------------------

def jackknife2(fileName):
    obs = readFile(fileName)
    data = []
    for h in range(0, len(obs)):
        for j in range(h + 1, len(obs)):
            for k in range(j + 1, len(obs)):
                data.append([obs[h], obs[j], obs[k]])
    
    min_dist = []
    min_dist_t = []
    t = []
    
    for i in range(0, len(data)):
        r0rdot0t0 = orbit(data[i])[1]
        close = closestEncounter(r0rdot0t0[0], r0rdot0t0[1], r0rdot0t0[2])
        min_dist.append(close[0][0])
        min_dist_t.append(close[0][1])
        t.append(close[1])
        
    print "Average closest encounter (current time (JD) = " + str(average(t)) + "):"
    print "\tDistance (AU):", str(average(min_dist)) + "+-" + str(error(min_dist))
    print "\tDays from now (JD):", str(average(min_dist_t)) + "+-" + str(error(min_dist_t))
    
    print "Average time (JD):", average(t)
    print "Average position vector (AU):", averageVec(r0)
    print "Average velocity vector (AU):", averageVec(rdot0)
    
        
jackknife("data12345678.txt")

hi
hi again
hi again again
r0 <0.210430, -1.437752, -0.532800>
rdot0 <0.833489, -0.527611, 0.138321>
r0 <0.217834, -1.390700, -0.520204>
rdot0 <0.842611, -0.459572, 0.136170>
r0 <0.220844, -1.371574, -0.515083>
rdot0 <0.846326, -0.431896, 0.135301>
r0 <0.222178, -1.363101, -0.512815>
rdot0 <0.847972, -0.419630, 0.134918>
r0 <0.222790, -1.359207, -0.511773>
rdot0 <0.848729, -0.413993, 0.134742>
r0 <0.223077, -1.357388, -0.511286>
rdot0 <0.849082, -0.411360, 0.134660>
r0 <0.223211, -1.356532, -0.511056>
rdot0 <0.849249, -0.410120, 0.134621>
r0 <0.223275, -1.356128, -0.510948>
rdot0 <0.849328, -0.409535, 0.134603>
r0 <0.223305, -1.355936, -0.510897>
rdot0 <0.849365, -0.409257, 0.134595>
r0 <0.223319, -1.355846, -0.510873>
rdot0 <0.849382, -0.409126, 0.134590>
r0 <0.223326, -1.355803, -0.510861>
rdot0 <0.849391, -0.409064, 0.134588>
r0 <0.223329, -1.355782, -0.510856>
rdot0 <0.849395, -0.409034, 0.134588>
r0 <0.223331, -1.355773, -0.510853>
rdot0 <0.849397, -0.409021, 0.134587>
r0 <0.22333

rdot0 <0.848213, -0.416679, 0.132889>
r0 <0.223134, -1.357022, -0.511188>
rdot0 <0.848213, -0.416679, 0.132889>
r0 <0.223134, -1.357022, -0.511188>
rdot0 <0.848213, -0.416679, 0.132889>
count 26
hi again again again
hi again again again
hi again again again
hi again again again
Middle Observation:
	position vector = <0.223134, -1.448382, 0.070787>
	velocity vector = <0.848213, -0.329435, 0.287669>
	light-corrected time (JD)= 2457943.69861

Classical orbital elements:
	semi-major axis (a) = 2.21035812634
	eccentricity (e) = 0.571345193046
	inclination (i) = 18.8067848675 °
	longitude of ascending node (big O) = 270.603984182 °
	argument of perihelion (small O) = 280.086550682 °
	true longitude (v) = 88.5206361529 °
hi again
hi again again
r0 <0.210555, -1.436959, -0.532587>
rdot0 <0.832913, -0.531058, 0.136904>
r0 <0.217790, -1.390981, -0.520279>
rdot0 <0.841828, -0.464502, 0.134826>
r0 <0.220706, -1.372452, -0.515318>
rdot0 <0.845430, -0.437642, 0.133999>
r0 <0.221985, -1.364324, -0.51

r0 <0.325354, -1.402878, -0.493610>
rdot0 <0.840449, -0.353109, 0.154865>
r0 <0.325354, -1.402878, -0.493610>
rdot0 <0.840449, -0.353109, 0.154865>
r0 <0.325354, -1.402878, -0.493610>
rdot0 <0.840449, -0.353109, 0.154865>
r0 <0.325354, -1.402878, -0.493610>
rdot0 <0.840449, -0.353109, 0.154865>
r0 <0.325354, -1.402878, -0.493610>
rdot0 <0.840449, -0.353109, 0.154865>
r0 <0.325354, -1.402878, -0.493610>
rdot0 <0.840449, -0.353109, 0.154865>
r0 <0.325354, -1.402878, -0.493610>
rdot0 <0.840449, -0.353109, 0.154865>
r0 <0.325354, -1.402878, -0.493610>
rdot0 <0.840449, -0.353109, 0.154865>
count 26
hi again again again
hi again again again
hi again again again
hi again again again
Middle Observation:
	position vector = <0.325354, -1.483462, 0.105154>
	velocity vector = <0.840449, -0.262369, 0.282545>
	light-corrected time (JD)= 2457950.72767

Classical orbital elements:
	semi-major axis (a) = 2.1799412335
	eccentricity (e) = 0.5575808326
	inclination (i) = 18.6315924988 °
	longitude of asce

r0 <0.440822, -1.442310, -0.470088>
rdot0 <0.826577, -0.295142, 0.172150>
r0 <0.440833, -1.442229, -0.470073>
rdot0 <0.826580, -0.295036, 0.172157>
r0 <0.440838, -1.442193, -0.470066>
rdot0 <0.826581, -0.294989, 0.172159>
r0 <0.440840, -1.442177, -0.470063>
rdot0 <0.826582, -0.294968, 0.172161>
r0 <0.440841, -1.442170, -0.470061>
rdot0 <0.826582, -0.294959, 0.172161>
r0 <0.440841, -1.442167, -0.470061>
rdot0 <0.826583, -0.294955, 0.172161>
r0 <0.440841, -1.442165, -0.470060>
rdot0 <0.826583, -0.294953, 0.172162>
r0 <0.440841, -1.442165, -0.470060>
rdot0 <0.826583, -0.294952, 0.172162>
r0 <0.440841, -1.442165, -0.470060>
rdot0 <0.826583, -0.294952, 0.172162>
r0 <0.440841, -1.442164, -0.470060>
rdot0 <0.826583, -0.294951, 0.172162>
r0 <0.440842, -1.442164, -0.470060>
rdot0 <0.826583, -0.294951, 0.172162>
r0 <0.440842, -1.442164, -0.470060>
rdot0 <0.826583, -0.294951, 0.172162>
r0 <0.440842, -1.442164, -0.470060>
rdot0 <0.826583, -0.294951, 0.172162>
r0 <0.440842, -1.442164, -0.470060>
rd

	argument of perihelion (small O) = 281.12247759 °
	true longitude (v) = 82.8874518811 °
hi again again
r0 <0.094771, -1.370510, -0.553257>
rdot0 <0.836881, -0.578213, 0.123308>
r0 <0.101048, -1.330482, -0.539536>
rdot0 <0.848300, -0.517038, 0.116953>
r0 <0.103585, -1.314307, -0.533991>
rdot0 <0.852940, -0.492122, 0.114447>
r0 <0.104702, -1.307180, -0.531548>
rdot0 <0.854990, -0.481109, 0.113353>
r0 <0.105213, -1.303922, -0.530431>
rdot0 <0.855928, -0.476069, 0.112856>
r0 <0.105450, -1.302409, -0.529912>
rdot0 <0.856363, -0.473725, 0.112625>
r0 <0.105561, -1.301700, -0.529669>
rdot0 <0.856567, -0.472628, 0.112517>
r0 <0.105614, -1.301368, -0.529555>
rdot0 <0.856663, -0.472113, 0.112467>
r0 <0.105638, -1.301211, -0.529502>
rdot0 <0.856708, -0.471870, 0.112443>
r0 <0.105650, -1.301137, -0.529476>
rdot0 <0.856730, -0.471756, 0.112431>
r0 <0.105655, -1.301102, -0.529464>
rdot0 <0.856740, -0.471702, 0.112426>
r0 <0.105658, -1.301086, -0.529459>
rdot0 <0.856744, -0.471677, 0.112424>
r0 <0.10

...

r0 <0.439283, -1.453775, -0.472304>
rdot0 <0.833358, -0.259295, 0.182410>
r0 <0.439283, -1.453775, -0.472304>
rdot0 <0.833358, -0.259295, 0.182410>
r0 <0.439283, -1.453775, -0.472304>
rdot0 <0.833358, -0.259295, 0.182410>
r0 <0.439283, -1.453775, -0.472304>
rdot0 <0.833358, -0.259295, 0.182410>
r0 <0.439283, -1.453775, -0.472304>
rdot0 <0.833358, -0.259295, 0.182410>
r0 <0.439283, -1.453775, -0.472304>
rdot0 <0.833358, -0.259295, 0.182410>
r0 <0.439283, -1.453775, -0.472304>
rdot0 <0.833358, -0.259295, 0.182410>
count 23
hi again again again
hi again again again
hi again again again
hi again again again
Middle Observation:
	position vector = <0.439283, -1.521685, 0.144948>
	velocity vector = <0.833358, -0.165340, 0.270499>
	light-corrected time (JD)= 2457958.72911

Classical orbital elements:
	semi-major axis (a) = 2.16206835938
	eccentricity (e) = 0.519111404252
	inclination (i) = 17.966169162 °
	longitude of ascending node (big O) = 269.709185092 °
	argument of perihelion (small O) =

r0 <0.105763, -1.300418, -0.529230>
rdot0 <0.855617, -0.479238, 0.110503>
r0 <0.105767, -1.300386, -0.529219>
rdot0 <0.855625, -0.479193, 0.110497>
r0 <0.105770, -1.300373, -0.529214>
rdot0 <0.855629, -0.479173, 0.110495>
r0 <0.105771, -1.300367, -0.529212>
rdot0 <0.855631, -0.479165, 0.110494>
r0 <0.105771, -1.300364, -0.529211>
rdot0 <0.855631, -0.479161, 0.110493>
r0 <0.105771, -1.300363, -0.529211>
rdot0 <0.855632, -0.479160, 0.110493>
r0 <0.105771, -1.300363, -0.529211>
rdot0 <0.855632, -0.479159, 0.110493>
r0 <0.105771, -1.300363, -0.529211>
rdot0 <0.855632, -0.479159, 0.110493>
r0 <0.105771, -1.300363, -0.529211>
rdot0 <0.855632, -0.479159, 0.110493>
r0 <0.105771, -1.300363, -0.529211>
rdot0 <0.855632, -0.479158, 0.110493>
r0 <0.105771, -1.300363, -0.529211>
rdot0 <0.855632, -0.479158, 0.110493>
r0 <0.105771, -1.300363, -0.529211>
rdot0 <0.855632, -0.479158, 0.110493>
r0 <0.105771, -1.300363, -0.529211>
rdot0 <0.855632, -0.479158, 0.110493>
r0 <0.105771, -1.300363, -0.529211>
rd

r0 <0.297977, -1.381753, -0.496747>
rdot0 <0.839954, -0.386903, 0.143254>
count 24
hi again again again
hi again again again
hi again again again
hi again again again
Middle Observation:
	position vector = <0.297977, -1.465328, 0.093873>
	velocity vector = <0.839954, -0.297993, 0.285334>
	light-corrected time (JD)= 2457948.74453

Classical orbital elements:
	semi-major axis (a) = 2.17797701714
	eccentricity (e) = 0.575577096481
	inclination (i) = 18.8633896733 °
	longitude of ascending node (big O) = 270.906654021 °
	argument of perihelion (small O) = 278.394063143 °
	true longitude (v) = 92.7799362766 °
hi again
hi again again
r0 <0.341819, -1.500100, -0.506623>
rdot0 <0.840532, -0.372742, 0.172557>
r0 <0.348511, -1.454507, -0.496797>
rdot0 <0.843627, -0.320216, 0.172291>
r0 <0.351020, -1.437414, -0.493113>
rdot0 <0.844791, -0.300503, 0.172196>
r0 <0.352032, -1.430518, -0.491627>
rdot0 <0.845261, -0.292548, 0.172158>
r0 <0.352452, -1.427656, -0.491010>
rdot0 <0.845456, -0.289246, 0.17

	true longitude (v) = 95.3842263568 °
hi again
hi again again
r0 <0.283002, -1.479781, -0.519808>
rdot0 <0.829949, -0.497859, 0.147228>
r0 <0.290732, -1.429176, -0.507903>
rdot0 <0.837038, -0.426898, 0.147581>
r0 <0.293804, -1.409069, -0.503173>
rdot0 <0.839843, -0.398814, 0.147696>
r0 <0.295130, -1.400390, -0.501131>
rdot0 <0.841052, -0.386711, 0.147742>
r0 <0.295722, -1.396512, -0.500219>
rdot0 <0.841591, -0.381307, 0.147762>
r0 <0.295991, -1.394754, -0.499806>
rdot0 <0.841836, -0.378857, 0.147770>
r0 <0.296113, -1.393951, -0.499617>
rdot0 <0.841948, -0.377739, 0.147774>
r0 <0.296170, -1.393583, -0.499530>
rdot0 <0.841999, -0.377227, 0.147776>
r0 <0.296195, -1.393414, -0.499490>
rdot0 <0.842022, -0.376992, 0.147777>
r0 <0.296207, -1.393337, -0.499472>
rdot0 <0.842033, -0.376884, 0.147777>
r0 <0.296213, -1.393302, -0.499464>
rdot0 <0.842038, -0.376834, 0.147777>
r0 <0.296215, -1.393285, -0.499460>
rdot0 <0.842040, -0.376811, 0.147777>
r0 <0.296216, -1.393278, -0.499458>
rdot0 <0.84204

rdot0 <0.838399, -0.338994, 0.162301>
r0 <0.353626, -1.419657, -0.489286>
rdot0 <0.839001, -0.329750, 0.162401>
r0 <0.354086, -1.416518, -0.488610>
rdot0 <0.839257, -0.325827, 0.162443>
r0 <0.354284, -1.415170, -0.488319>
rdot0 <0.839367, -0.324140, 0.162461>
r0 <0.354370, -1.414587, -0.488193>
rdot0 <0.839414, -0.323411, 0.162469>
r0 <0.354407, -1.414334, -0.488139>
rdot0 <0.839435, -0.323096, 0.162473>
r0 <0.354423, -1.414225, -0.488115>
rdot0 <0.839444, -0.322959, 0.162474>
r0 <0.354430, -1.414177, -0.488105>
rdot0 <0.839448, -0.322900, 0.162475>
r0 <0.354433, -1.414157, -0.488101>
rdot0 <0.839449, -0.322874, 0.162475>
r0 <0.354434, -1.414148, -0.488099>
rdot0 <0.839450, -0.322863, 0.162475>
r0 <0.354435, -1.414144, -0.488098>
rdot0 <0.839450, -0.322858, 0.162475>
r0 <0.354435, -1.414142, -0.488098>
rdot0 <0.839450, -0.322856, 0.162475>
r0 <0.354435, -1.414142, -0.488097>
rdot0 <0.839450, -0.322855, 0.162475>
r0 <0.354435, -1.414141, -0.488097>
rdot0 <0.839451, -0.322854, 0.162475>


In [ ]:
#2457943.69861
#p = 2.989827044354738E-01, -9.673318697934900E-01, -1.039527525330058E-04
#v = 1.616669804555460E-02, 4.953549549521496E-03, 3.266014706582907E-07

#

